In [1]:
import models

In [2]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=1,
    arrival_rates=[4, 5],
    service_rates=[12, 13],
    thetas=[[None, 2], [1, None]],
    infty=15,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:09<00:00, 2183.89it/s]           


{'Markov Chain': [0.18902347102777123, 0.398686645719244],
 'Simulation': [0.19387962688300975, 0.4096967517870087]}

In [3]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=1,
    arrival_rates=[3, 5],
    service_rates=[12, 13],
    thetas=[[None, 2], [4, None]],
    infty=15,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:08<00:00, 2257.28it/s]           


{'Markov Chain': [0.18089660366279517, 0.27335350278687814],
 'Simulation': [0.18272704820161545, 0.27482538387242944]}

In [4]:
models.compare_mc_to_sim_sojourn(
    num_classes=3,
    num_servers=1,
    arrival_rates=[0.5, 1.3, 0.6],
    service_rates=[9, 9, 8],
    thetas=[[None, 1, 0.6], [1.1, None, 0.7], [1.2, 1, None]],
    infty=8,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:02<00:00, 7289.59it/s]           


{'Markov Chain': [0.1204092506720846, 0.1529471254170382, 0.19299607180496842],
 'Simulation': [0.12068273808580142, 0.154405733903177, 0.19563222490353108]}

In [5]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=2,
    arrival_rates=[3, 4],
    service_rates=[6.5, 7.5],
    thetas=[[None, 2], [4, None]],
    infty=15,
    max_simulation_time=50000,
    warmup=200,
)

100%|██████████| 50000.0/50000 [00:16<00:00, 2950.46it/s]           


{'Markov Chain': [0.1794581385915189, 0.21367804897072376],
 'Simulation': [0.17848945316697193, 0.21354310076383318]}

In [6]:
models.compare_mc_to_sim_sojourn(
    num_classes=3,
    num_servers=2,
    arrival_rates=[1, 0.8, 0.5],
    service_rates=[3, 2.5, 2],
    thetas=[[None, 1, 0.5], [0.5, None, 0.5], [1, 0.7, None]],
    infty=8,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:03<00:00, 6636.28it/s]           


{'Markov Chain': [0.352441377466164, 0.5031135932339846, 0.6393170884752223],
 'Simulation': [0.35245868593884844, 0.5075393946421101, 0.6364805806806387]}